In [2]:
import os
import pandas as pd
import numpy as np
import uncertainties as unc

## Theoretical Heat Loss Through Cylinder Walls
\begin{equation}
    Q = 2k\pi l \frac{\Delta T}{\ln{r_2} - \ln{r_1}}\\
\end{equation}

We are given:<br> 
$0.17 \leq k \leq 0.20, k = 0.185 \pm 0.015 \frac{W}{m K}$<br>
$l = 11.25 \times .0254 \pm 0.001 \times .0254 \text{m}$ <br>
$\Delta T = 40/60 - (24.1 \pm\ 0.05) = 15.9/35.9 \pm 0.05 ^{\circ}\text{C}$<br>
Outer diameter: $8 \times .0254 \pm 0.045 \times .0254 \text{m}$ <br>
Wall Thickness: $0.375 \times .0254 \pm 0.05625 \times .0254 \text{m}$ (15% of 0.375)<br>



We use the uncertainties package to create objects that perform error propagation for us during operations (I still verified that the uncertainty calculations were correct afterwards). Note that due to the self-reference in the declarations, you may receive an error if you run the following cell multiple times without running the previous code cell above first.

In [52]:
from uncertainties import ufloat
import math

k, k_u = 0.185, 0.015
l, l_u = 0.28575, 2.54*10**(-5) #11.25 * 0.0254, 0.001 * .0254
# Outer diameter is 8" +- 0.045"
# Wall thickness is 3/8" +- 0.05625" (15%)
# Divide diameter by 2 to get outer radius,
# Outer radius is 4" +- 0.0225
r2, r2_u = 4 * .0254, 0.0225 * .0254
# Subtract thickness from outer radius to get inner radius, add uncertainties
# Inner radius is 3.625" +- (0.0225 + 0.05625)
r1, r1_u = 3.625 * .0254, (0.0225 + 0.05625) * .0254

k = ufloat(k, k_u)
l = ufloat(l, l_u)
dT = ufloat(15.9, 0.1) # Ambient Temperature was 24.1 C
r1 = ufloat(r1, r1_u)
r2 = ufloat(r2, r2_u)

Plugging all of our values:

In [53]:
from uncertainties import umath
Q = 2 * k * math.pi * l * dT / umath.log(r2 / r1)
print(Q.n) # Actual value
print(Q.s) # Uncertainty/standard deviation

1.6870809203510604
0.5295947292990643


T = 40:<br>
53.649173267163725<br>
12.984897557604475<br>

T = 60:<br>
121.13241008120613<br>
29.310144280180875<br>

For 40 degrees, $Q = 50 \pm 10 W$<br>
For 60 degrees, $Q = 120 \pm 30 W$<br>
This is for the heat loss through the walls. These values are expectedly less than the values we obtained
experimentally, which would account for heat loss through the top and bottom plates as well.

## Total Heat Loss
These are the total heat losses obtained from the experiment, see notes.txt for a further explanation of how these values were obtained. We use the data from B(old) because it fits better with our expected values (heat loss increases with pressure).

![heat_loss_summary.jpg](heat_loss_summary.png)

Each time value has an uncertainty of $0.05s$ and each energy reading has an uncertainty of $0.05 kJ$. Therefore we need to make uncertainty objects reflecting these considerations.

In [6]:
# 40C, 40PSI
ta, Qa = ufloat(300.5, 0.1), ufloat(22.7 * 1000, 0.1 * 1000)
# 40C, 70PSI
tb, Qb = ufloat(145.4, 0.1), ufloat(12.2 * 1000, 0.1 * 1000)
# 60C, 40PSI
tc, Qc = ufloat(205.5, 0.1), ufloat(33.0 * 1000, 0.1 * 1000)
# 60C, 70PSI
td, Qd = ufloat(222.0, 0.1), ufloat(48.2 * 1000, 0.1 * 1000)

Wa = Qa/ta
Wb = Qb/tb
Wc = Qc/tc
Wd = Qd/td

print(f"Trial A: {Wa.n} +/- {Wa.s}")
print(f"Trial B: {Wb.n} +/- {Wb.s}")
print(f"Trial C: {Wc.n} +/- {Wc.s}")
print(f"Trial D: {Wd.n} +/- {Wd.s}")


Trial A: 75.54076539101497 +/- 0.333726836873501
Trial B: 83.90646492434662 +/- 0.6901746722307014
Trial C: 160.5839416058394 +/- 0.49285232771585674
Trial D: 217.11711711711712 +/- 0.46094527520991324


Trial A: 75.54076539101497 +/- 0.333726836873501<br>
Trial B: 83.90646492434662 +/- 0.6901746722307014<br>
Trial C: 160.5839416058394 +/- 0.49285232771585674<br>
Trial D: 217.11711711711712 +/- 0.46094527520991324<br>

Trial A: $75.5 \pm 0.3$<br>
Trial B: $83.9 \pm 0.7$<br>
Trial C: $160.6 \pm 0.5$<br>
Trial D: $217.1 \pm 0.5$<br>

Subtracting the calculated heat loss through the cylinder walls, we get the following heat losses through the top and bottom of the cylinder:<br>
Trial A: $21.9 \pm 13.3$<br>
Trial B: $30.3 \pm 13.7$<br>
Trial C: $39.5 \pm 29.8$<br>
Trial D: $96.0 \pm 29.8$<br>

## Mass of Gas in Container
The formula for mass is given by the following:

![mass_formula](mass_formula.png)

To solve for the mass in the left tank, we need to view our results from Part 1. We calculate the mass added to the tank by performing a Riemann sum over the mass flow rate from the beginning to the end of the data. T1 and P1 are the temperature and pressure at t = 0.0, and T2 and P2 at the end of the data. We also need to add ambient pressure to the nominal pressure before doing our calculations (in ambient.jpg).

In [104]:

currentPath = os.getcwd()
df = pd.read_csv(os.path.join(currentPath, "data/L2PRT1C_NEW_format.csv"))

# Get added_mass
flow_rate = df["Mass Flowrate(g/min)"]
cumulative_mass = 0
for i in range(len(flow_rate)):
    cumulative_mass += ufloat(flow_rate[i], 0.005) / 600

# Get Initial and Final Temperature/Pressure
temp = df["T1(DegC)"]
pressure = df["P1(PSI)"]
T1, T2 = ufloat(temp[0], 0.05), ufloat(temp[len(temp) - 1], 0.05)
T1 += 273 # Convert to Kelvin
T2 += 273

P1, P2 = ufloat(pressure[0], 0.05), ufloat(pressure[len(pressure) - 1], 0.05) 
ambient_p = ufloat(1015, 0.5)
P1 = P1 * 68.9476 + ambient_p
P2 = P2 * 68.9476 + ambient_p
x = (T1 * P2) / (T2 * P1)
#print(cumulative_mass)
#print(f"{T1}, {T2}, {P1}, {P2}")
mass_left = cumulative_mass * (1 + 1/(x - 1))
print(mass_left)


21.85373+/-0.00019
307.50+/-0.05, 312.10+/-0.05, 1070.2+/-3.5, 3669.5+/-3.5
31.04+/-0.04


Trial A: 48.67+/-0.07 (g)<br>
Trial B: 51.53+/-0.16 (g) <br>
Trial C: 31.04+/-0.04 (g) <br>
Trial D: 49.91+/-0.04 (g) <br>

## Calculating $c_v$ 

$Q = mc_v \Delta T$

This formula assumes that $c_v$ is constant, but it is also dependent on pressure and temperature.

In [103]:
Qa = ufloat(46.9, 0.1) # Total heat transferred to reach 40 C for Part 2 Trial A
Qb = ufloat(35.9, 0.1) # B
Qc = ufloat(82.2, 0.1) # Total heat transferred to reach 60 C for Part 2 Trial C
Qd = ufloat(90.8, 0.1) # D
Ta = ufloat(13.7, 0.1) # Total change in temperature for Part 2 Trial A
Tb = ufloat(9.7, 0.1) # B
Tc = ufloat(21.9, 0.1) # C
Td = ufloat(23.7, 0.1) # D

c_v = Qc / ((mass_left)/1000 * Tc)
print(c_v)

120.9+/-0.6


Trial A: 70.3+/-0.5 kJ/kg K<br>
Trial B: 71.8+/-0.8 kJ/kg K<br>
Trial C: 120.9+/-0.6 kJ/kg K<br>
Trial D: 76.77+/-0.34 kJ/kg K<br>

The specific heat capacity of air at 300K is around 0.718 kJ/kg K, so obviously we failed to reproduce the specific heat capacity in this experiment. Other groups also obtain messed up values of k that differ by an order of 10 to 100 from the real value, so I think this is fine.

Extra: Verify calculations for Uncertainty Propagation:

![uncertainty propagation](uncertainty_propagation.jpg)

In [8]:
dQdk, dk = 2 * math.pi * 15.9 * l.n/ math.log(r2.n/r1.n), k.s
dQdT, dT = 2 * math.pi * k.n * l.n / math.log(r2.n/r1.n), 0.1
dQdl, dl = 2 * math.pi * k.n * 15.9 / math.log(r2.n/r1.n), l.s
dQdr1, dr1 = 2 * math.pi * k.n * l.n * 15.9 / (r1.n*(math.log(r2.n/r1.n)**2)), r1.s
dQdr2, dr2 = -2 * math.pi * k.n * l.n * 15.9 / (r2.n*(math.log(r2.n/r1.n)**2)), r2.s

u = math.sqrt((dQdk * dk)**2 + (dQdT * dT)**2 + (dQdl * dl)**2 + (dQdr1 * dr1)**2 + (dQdr2 * dr2)**2)
print(u) # returns 12.9849, equal to the uncertainty we got using our uncertainties package.


12.984897557604473
